# 配送計画モデルVRP_D_1M1_TWの実行例
使用可能な機能：距離，容量制約1M1, 時間枠

## ランダムな問題例の作成

In [ ]:
n=10 #客数
demand_dim=2 # 需要の次元

import random
random.seed(1)

C={} # 客の座標を保存する辞書
TW={} # 客の時間枠を保存する辞書
demand={} # 客の要求量(正の値は集荷，負の値は配達)
S={} # 客のサービス時間

for i in range(n):
    name="c"+str(i)
    x=random.randint(-500,500)
    y=random.randint(-500,500)
    C[name]=(x,y)
    e=random.randint(0,3000)
    l=e+random.randint(0,4000-e)
    TW[name]=(e,l)

    demand[name]=tuple(random.randint(-5,5) for k in range(demand_dim))
    S[name]=random.randint(10,20)
        
C["depot"]=(0,0)

In [ ]:
print("Customer",C)
print("Time Window",TW)
print("Demand",demand)
print("Service Time",S)

## 車両数と容量の設定

In [ ]:
m=10 # number of vehicles
capacity=tuple(30 for k in range(demand_dim)) # capacity of vehicle

## 問題例の描画

In [ ]:
import networkx as nx
import matplotlib.pyplot as plt

fig=plt.figure(figsize=(7,7))

G=nx.DiGraph()
nx.draw_networkx(G,pos=C,nodelist=[i for i in C if i != "depot"],node_color="y",node_size=50,with_labels=True,edge_color="k",width=1)
nx.draw_networkx(G,pos=C,nodelist=["depot"],node_color="blue",node_shape='s',alpha=0.5,node_size=100,with_labels=True,edge_color="k",width=1)

plt.show()

In [ ]:
def distance(t1,t2):
    return ((t1[0]-t2[0])**2+(t1[1]-t2[1])**2)**(0.5)

In [ ]:
import sys
sys.path.append('..')

from vrplib.vrp_d_1m1_t_model import *

## 配送計画ソルバーの実行

In [ ]:
import sys
sys.path.append('..')

#from vrplib.vrp_d_1m1_t_model import *

model = Model("example-vrp_d_1m1_tw") # モデルインスタンスの生成

# 客インスタンスの生成
for i in C:
    if i == "depot":
        continue
    model += Customer(i,demand=demand[i],timewindow=TW[i],servicetime=S[i])

# 車両インスタンスの生成
for k in range(m):
    model += Vehicle("v"+str(k),capacity=capacity)
    
# 枝インスタンスの生成
for i in C:
    for j in C:
        if i!=j:
            dist = time = distance(C[i],C[j])
            model += Edge(i,j,dist,time)

# 最適化の実行
obj=model.optimize(IterLimit=5,TimeLimit=11,Verbose=True,OutputFlag=False)

# 得られた解の表示
print("objective value =",obj)
for v in model.vehiclesL:
    print()
    print(v)
    print(list(map(lambda x:x.name,v.routing[1:-1])))    
    for iv in map(lambda x:x.name,v.routing[1:-1]):
        print(iv)

## 得られた解の描画

In [ ]:
import networkx as nx
import matplotlib.pyplot as plt
G=nx.DiGraph()
for v in model.vehiclesL:
    prev=None
    for j in v.routing[1:-1]:
        if prev == None:                
            G.add_edge("depot",j.name)
        else:
            G.add_edge(prev.name,j.name)
        prev= j 
    G.add_edge(prev.name,"depot")
    
fig=plt.figure(figsize=(9,9))    
nx.draw_networkx(G,pos=C,nodelist=[i for i in G.nodes() if i != "depot"],node_color="yellow",node_size=50,with_labels=True,edge_color="k",width=1)
nx.draw_networkx(G,pos=C,nodelist=["depot"],node_color="blue",node_shape='s',alpha=0.5,node_size=50,with_labels=True,edge_color="k",width=1)
plt.show()

## 得られた解の詳細な情報

In [ ]:
for v in model.vehiclesL:
    print()
    print(v)
    print("v.stat:",v.stat)
    keys=v.routing[1].stat.keys()
    print("customer,",", ".join(keys))
    for c in v.routing[1:-1]:
        print(c.name+", ",end="")
        for k in keys:
            print(c.stat[k],end=",")
        print()